# <div align="center"><b> Ejemplo de fine-tuning </b></div>

<div align="right">

<!-- [![Binder](http://mybinder.org/badge.svg)](https://mybinder.org/) -->
<!-- [![nbviewer](https://img.shields.io/badge/render-nbviewer-orange?logo=Jupyter)](https://nbviewer.org/)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/) -->

</div>

* * *

<style>
/* Limitar la altura de las celdas de salida en html */
.jp-OutputArea.jp-Cell-outputArea {
    max-height: 500px;
}
</style>

🛻 <em><font color='MediumSeaGreen'>  Instalaciones: </font></em> 🛻


Este notebook utiliza [Poetry](https://python-poetry.org/) para la gestión de dependencias.
Primero instala Poetry siguiendo las instrucciones de su [documentación oficial](https://python-poetry.org/docs/#installation).
Luego ejecuta el siguiente comando para instalar las dependencias necesarias y activar el entorno virtual:

- Bash:

```bash
poetry install
eval $(poetry env activate)
```

- PowerShell:

```powershell
poetry install
Invoke-Expression (poetry env activate)
```

> 📝 <em><font color='Gray'>Nota:</font></em> Para agregar `pytorch` utilizando Poetry, se utiliza el siguiente comando:
> ```bash
> # Más info: https://github.com/python-poetry/poetry/issues/6409
> poetry source add --priority explicit pytorch_gpu https://download.pytorch.org/whl/cu128 # Seleccionar la wheel adecuada para tu GPU
> poetry add --source pytorch_gpu torch torchvision 
> ```

✋ <em><font color='DodgerBlue'>Importaciones:</font></em> ✋

In [1]:
# Recarga automática de módulos en Jupyter Notebook
%reload_ext autoreload
%autoreload 2

from datasets import Dataset
from loguru import logger
from tqdm import tqdm
import torch

import mlflow
import requests
import os

import pandas as pd

# Modulos propios
from vision_transformer.dataset import load_huggingface_dataset
from vision_transformer.config import MLFLOW_URL, MODELS_DIR, PREFECT_URL, DATASET_NAME, DATASET_VERSION, HISTORY_FILENAME

2025-06-18 19:34:28.564 | INFO     | vision_transformer.config:<module>:15 - PROJ_ROOT path is: E:\Documentos\Git Repositories\vision-transformer
2025-06-18 19:34:28.565 | INFO     | vision_transformer.config:<module>:19 - Actual environment is: dev


🔧 <em><font color='tomato'>Configuraciones:</font></em> 🔧


In [2]:
BATCH_SIZE = 16 
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"  # Establece el dispositivo.
print(f"Dispositivo actual: {DEVICE}")
# # torch.set_float32_matmul_precision('highest') # Optimización: Establece la precisión de las multiplicaciones de matrices de punto flotante de 32 bits en 'más alta'.
# torch.set_float32_matmul_precision('high') # Optimización: Establece la precisión de las multiplicaciones de matrices de punto flotante de 32 bits en 'alta'.
# # torch.set_float32_matmul_precision('medium') # Optimización: Establece la precisión de las multiplicaciones de matrices de punto flotante de 32 bits en 'media'.
# # torch.backends.cudnn.benchmark = True # Optimización: Para redes CNN (pero como se usa una capa convolucional, se establece en True).

MODEL_NAME = "vit-base"  # Nombre del modelo de Hugging Face.
MODEL_FOLDER = MODELS_DIR / "vit-base"
CHECKPOINT = "google/vit-base-patch16-224-in21k"

# MLflow: Configuración de la URI de seguimiento
try:
    response = requests.get(MLFLOW_URL)
    response.raise_for_status()  # Verifica si la solicitud fue exitosa.
    logger.success("Conexión a MLflow establecida correctamente.")
    os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_URL  # Configura la URI de seguimiento de MLflow.
    os.environ["MLFLOW_EXPERIMENT_NAME"] = CHECKPOINT.replace("/", "_")  # Configura el nombre del experimento de MLflow.
    os.environ["MLFLOW_TAGS"] = '{"model_family": "swinv2"}'
except Exception as e:
    logger.error(f"Error al conectar con MLflow. Tienes levantado el servidor de MLflow?")
    raise SystemExit(f"Error al conectar con MLflow: {e}")

# Prefect: Configuración de Prefect
try:
    response = requests.get(PREFECT_URL)
    response.raise_for_status()  # Verifica si la solicitud fue exitosa.
    logger.success("Conexión a Prefect establecida correctamente.")
except Exception as e:
    logger.error(f"Error al conectar con Prefect. Tienes levantado el servidor de Prefect?")
    raise SystemExit(f"Error al conectar con Prefect: {e}")

Dispositivo actual: cuda
2025-06-18 19:34:31.240 | SUCCESS  | __main__:<module>:17 - Conexión a MLflow establecida correctamente.
2025-06-18 19:34:33.261 | SUCCESS  | __main__:<module>:29 - Conexión a Prefect establecida correctamente.


<div align="center">✨Datos del proyecto:✨</div>

<p></p>

<div align="center">

| Subtitulo       | *Fine-tuning* del modelo swimv2 sobre el dataset EuroSAT                                                                       |
| --------------- | -------------------------------------------------------------------------------------------------------------------------------------- |
| **Descrpción**  | <small>Análisis exploratorio del proceso de *fine-tuning* del swimv2 sobre el EuroSAT<br/>- *Tarea:* `Clasificación`<br/>- *Modelo*: `swimv2`<br/> - *Dataset*: `EuroSAT` </small>|
| **Autor** | <small>[Nombre] ([correo]) </small>                                                                                                 |

</div>

## Tabla de contenidos
1. [Carga de datos](#carga-de-datos)

## 1. Carga de datos <a name="carga-de-datos"></a>

```python

In [3]:
dataset = load_huggingface_dataset()

2025-06-18 19:34:33.303 | INFO     | vision_transformer.dataset:load_huggingface_dataset:441 - Cargando el dataset procesado...
2025-06-18 19:34:33.588 | INFO     | vision_transformer.dataset:load_huggingface_dataset:453 - El dataset contiene múltiples conjuntos (train, test, val). Cargando todos...


Resolving data files:   0%|          | 0/24300 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/2700 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 24300
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 2700
    })
})

## 2. Procesamiento de datos <a name="procesamiento-de-datos"></a>

In [5]:
labels = dataset["train"].features["label"].names
id2label = {id: label for id, label in enumerate(dataset["train"].features["label"].names)}
label2id = {label: id for id, label in id2label.items()}

In [6]:
# TODO: Comentar este bloque de prueba cuando se use todo el dataset.
from vision_transformer.config import RANDOM_SEED

dataset['train'] = dataset['train'].shuffle(seed=RANDOM_SEED).select(range(100))
dataset['test'] = dataset['test'].shuffle(seed=RANDOM_SEED).select(range(80))
test_image = dataset['test'][0]['image']

In [7]:
print(dataset['train'][0])

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=64x64 at 0x1FD707D8410>, 'label': 0}


In [8]:
from transformers import AutoImageProcessor

checkpoint = "google/vit-base-patch16-224-in21k"
image_processor = AutoImageProcessor.from_pretrained(checkpoint, use_fast=True)

In [9]:
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)

size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)

_transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize])

In [10]:
def transforms(examples):
    examples["pixel_values"] = [_transforms(img) for img in examples["image"]]
    del examples["image"]
    return examples

In [11]:
dataset = dataset.with_transform(transforms)

In [12]:
print(dataset['train'][0])
print(dataset['train'][0]['pixel_values'].shape)

{'label': 0, 'pixel_values': tensor([[[ 0.6863,  0.6863,  0.6863,  ...,  0.2000,  0.2000,  0.2000],
         [ 0.6863,  0.6863,  0.6863,  ...,  0.2000,  0.2000,  0.2000],
         [ 0.6863,  0.6863,  0.6863,  ...,  0.2000,  0.2000,  0.2000],
         ...,
         [ 0.7569,  0.7569,  0.7569,  ...,  0.1922,  0.1922,  0.1922],
         [ 0.7569,  0.7569,  0.7569,  ...,  0.1922,  0.1922,  0.1922],
         [ 0.7569,  0.7569,  0.7569,  ...,  0.1922,  0.1922,  0.1922]],

        [[ 0.4588,  0.4588,  0.4588,  ..., -0.0118, -0.0118, -0.0118],
         [ 0.4588,  0.4588,  0.4588,  ..., -0.0118, -0.0118, -0.0118],
         [ 0.4588,  0.4588,  0.4588,  ..., -0.0118, -0.0118, -0.0118],
         ...,
         [ 0.4510,  0.4510,  0.4510,  ...,  0.0588,  0.0588,  0.0588],
         [ 0.4510,  0.4510,  0.4510,  ...,  0.0588,  0.0588,  0.0588],
         [ 0.4510,  0.4510,  0.4510,  ...,  0.0588,  0.0588,  0.0588]],

        [[ 0.3490,  0.3490,  0.3490,  ..., -0.0667, -0.0667, -0.0667],
         [ 0.349

In [13]:
# # IMPORTANTE: Si se está en un .py, se debe ejecutar dentro del bloque `if __name__ == "__main__":`
# # if __name__ == "__main__":
# #     benchmark_dataloader(...)

import multiprocessing
import time
from torch.utils.data import DataLoader
import copy
from datasets import load_dataset  # solo por referencia, ya lo tenés cargado
from tqdm import tqdm
import torch

simulation_dataset = copy.deepcopy(dataset['train'])

# Asegurarse de que esté en formato PyTorch
simulation_dataset.set_format(type="torch", columns=["image", "label"])

CPU_COUNT = multiprocessing.cpu_count()

def benchmark_dataloader(dataset, batch_size=BATCH_SIZE, cpu_count=CPU_COUNT):
    num_workers_list = [i for i in range(0, cpu_count, 2)]  # De 0 a CPU_COUNT en pasos de 2
    logger.info(f"🧪 Benchmark con {CPU_COUNT} CPUs lógicos")
    for num_workers in num_workers_list:
        loader = DataLoader(dataset, batch_size=batch_size, num_workers=num_workers)

        start = time.time()
        for _ in tqdm(loader, desc=f"num_workers={num_workers:>2}", leave=False):
            pass  # Solo iteramos para medir el tiempo
        end = time.time()
        
        logger.info(f"🔹 num_workers = {num_workers:>2} → tiempo = {end - start:.2f} s")


# # Ejecutar benchmark
# # benchmark_dataloader(simulation_dataset, batch_size=BATCH_SIZE)

In [14]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [15]:
import evaluate

accuracy = evaluate.load("accuracy")

In [16]:
import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [17]:
from transformers import EarlyStoppingCallback, TrainerCallback
from transformers.integrations import MLflowCallback

# mlflowcallback = MLflowCallback(
#     tracking_uri="http://localhost:5000",  # Cambia esto si tu servidor MLflow está en otro lugar.
#     experiment_name="vision_transformer_experiment",
#     save_model=False,  # No guardar el modelo automáticamente, se guardará manualmente.
# )


class EmptyCacheCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, **kwargs):
        torch.cuda.empty_cache()


callback_list = [
    EarlyStoppingCallback(
        early_stopping_patience=2
    ),  # Usar con: metric_for_best_model = "eval_accuracy" o "eval_loss" en Trainer.
    MLflowCallback()
]

In [18]:
from transformers import AutoModelForImageClassification

model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
print(model)

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)
            (intermed

In [20]:
from transformers import TrainingArguments

from vision_transformer.config import MODELS_DIR

# NOTA: Un "step" es un "batch".
training_args = TrainingArguments(
    output_dir=MODELS_DIR / MODEL_FOLDER,  # Directorio de salida para los modelos entrenados.
    overwrite_output_dir=True,  # Sobrescribe el directorio de salida si ya existe.
    eval_strategy="epoch",  # Estrategia de evaluación: evalúa al final de cada época.
    per_device_train_batch_size=BATCH_SIZE, # Tamaño del lote por dispositivo durante el entrenamiento. Oportunidad de optimización.
    per_device_eval_batch_size=BATCH_SIZE, # Tamaño del lote por dispositivo durante la evaluación. Oportunidad de optimización.
    # gradient_accumulation_steps=4,    # Acumulación de gradientes: acumula gradientes antes de hacer un optimizer.step() para simular un tamaño de lote más grande. Por defecto es 1.
                                        # Oportunidad de optimización.
    # eval_accumulation_steps=4, # Acumulación de gradientes durante la evaluación: Sin establecer, acumula todas las predicciones antes de calcular las métricas (consume mas memoria). 
                                 # Oportunidad de optimización.
    # eval_delay=10, # Retraso de evaluación: espera 10 pasos antes de realizar la primera evaluación.
    # torch_empty_cache_steps=len(dataset['train']) // BATCH_SIZE   # Pasos para vaciar la caché de PyTorch: vacía la caché cada vez que se completa un paso de entrenamiento.
                                                                    # También se puede utilizar un callback.
    learning_rate=5e-5,  # Tasa de aprendizaje: tasa de aprendizaje inicial para el optimizador AdamW | Si se utiliza scheduler, puede ser mayor (por ejemplo, 1e-4, 2e-4 o 1e-3).
                         # Oportunidad de optimización.
    # weight_decay=0.01,  # Decaimiento del peso: regularización L2 para evitar el sobreajuste (se aplica a todos menos a los bias y capas de normalización).
                        # Oportunidad de optimización.
    # adam_beta1=0.9, # Beta1 para el optimizador AdamW: primer momento. Oportunidad de optimización.
    # adam_beta2=0.999, # Beta2 para el optimizador AdamW: segundo momento. Oportunidad de optimización.
    # adam_epsilon=1e-8, # Epsilon para el optimizador AdamW: evita la división por cero. Oportunidad de optimización.
    num_train_epochs=5,  # Número de épocas de entrenamiento: número total de épocas para entrenar el modelo.
    # lr_scheduler_type="reduce_lr_on_plateau", # Decaimiento de la tasa de aprendizaje: reduce la tasa de aprendizaje cuando la métrica de evaluación no mejora. 
                                                # Se puede user "linear", "cosine", "cosine_with_restarts", "polynomial", "constant", "constant_with_warmup".
                                                # "cosine_with_restarts" es útil para ciclos de entrenamiento, cuando se estanca en un mínimo local (o suponemos que se estanca).
                                                # Oportunidad de optimización.
    # lr_scheduler_kwargs={
    #     "factor": 0.1,
    #     "patience": 10,
    #     "threshold": 1e-4,
    #     "min_lr": 0,
    #     "mode": "min",
    #     "verbose": True
    # }, # Argumentos adicionales para el planificador de tasa de aprendizaje. Oportunidad de optimización.
    warmup_ratio=0.1,   # Proporción de calentamiento: aumenta la tasa de aprendizaje linealmente desde 0 hasta la tasa 
                          # de aprendizaje inicial durante el porcentaje especificado del total
                          # de pasos de entrenamiento (por ejemplo, 0.1 significa que la tasa de aprendizaje se calentará durante el 10% de los pasos de entrenamiento. 
                          # Ej: 10 épocas con 100 batches (o sea 1000 pasos), la tasa de aprendizaje se calentará durante los primeros 100 pasos (10% de 1000)). 
                          # NO APLICA PARA "reduce_lr_on_plateau" o "constant". Oportunidad de optimización.
    # log_level="info", # Nivel de registro: establece el nivel de registro para el entrenamiento 
                        # (puede ser "debug", "info", "warning", "error" o "critical"). Por defecto es "passive"
                        # que loguea lo actual de la librería transformers ("warning")
    # logging_steps=10,   # Pasos de registro: número de pasos entre registros. Si es 10, se registra cada 10 pasos.
    save_strategy="best", # Estrategia de guardado: puede ser "no", "epoch", "steps" o "best". Si es "steps", se guarda cada cierto número de pasos definido en save_steps.
    save_total_limit=1, # Limite de guardado total: número máximo de modelos guardados. Si se supera, se eliminan los más antiguos.
    logging_strategy="epoch",  # Estrategia de registro: puede ser "no", "epoch" o "steps". Si es "steps", se registra cada cierto número de pasos definido en logging_steps.
    seed=RANDOM_SEED,  # Semilla para la reproducibilidad: establece una semilla para la generación de números aleatorios.
    # bf16=True,    # Habilita el entrenamiento en punto flotante de 16 bits (FP16): reduce el uso de memoria y acelera el 
                    # entrenamiento en GPUs compatibles (Ampere o posteriores). Sino utiliza 32.
    # fp16=True, # Habilita el entrenamiento en punto flotante de 16 bits (FP16): reduce el uso de memoria y acelera el 
                 # entrenamiento en GPUs compatibles (Volta o posteriores). Sino utiliza 32.
    # tf32=True, # Habilita el entrenamiento en punto flotante de 32 bits (TF32): acelera el entrenamiento en 
                 # GPUs Ampere o posteriores. Relacionado con matmul de PyTorch.
    # dataloader_num_workers=0,  # Número de trabajadores para el cargador de datos: número de procesos que se utilizan para cargar los datos en paralelo. 
                               # Por defecto 0 (thread principal). NOTA: En notebook, hay que usar 0. En .py, se puede cambiar, pero se debe poner
                               # el código de entrenamiento dentro de un bloque `if __name__ == "__main__":` para evitar problemas de multiproceso.
                               # Esto en Windows.
    # run_name="vision_transformer_experiment", # Nombre de la ejecución: nombre de la ejecución para el seguimiento de experimentos (MLflow, WandB, etc.). 
                                                # Si no se especifica, se usa output_dir.
    remove_unused_columns=False,  # Elimina las columnas no utilizadas: si es True, elimina las columnas que no se utilizan en el modelo. 
                                  # Si es False, conserva todas las columnas.
                                  # Para este caso debe ser obligatoriamente False, ya que se usa una columna personalizada "pixel_values" para las imágenes.
    load_best_model_at_end=True,  # Carga el mejor modelo al final del entrenamiento: carga el modelo con la mejor métrica de evaluación al final del entrenamiento.
    metric_for_best_model="accuracy",   # Métrica para el mejor modelo: métrica que se utiliza para determinar el mejor modelo. 
                                        # Debe ser el nombre de una métrica que retorne el "evaluator".
                                        # Se usa en conjunto con `load_best_model_at_end=True`
    # optim="adamw_torch",  # Optimizador: optimizador a utilizar. Puede ser "adamw_torch", "adamw_hf" o "sgd". Por defecto es "adamw_torch".
                            # Lista completa: https://github.com/huggingface/transformers/blob/main/src/transformers/training_args.py#L143
    # optim_kwargs={} # Argumentos adicionales para el optimizador: diccionario con argumentos adicionales para el optimizador.
    # dataloader_persistent_workers=True,  # Habilita trabajadores persistentes para el cargador de datos: si es True, los trabajadores se mantienen 
                                         # activos entre épocas (consume más memoria RAM). Por defecto es False.
    # dataloader_prefetch_factor=2,   # Factor de prefetch para el cargador de datos: número de lotes que se prefetchean en segundo plano. 
                                    # Si es 2, significa que habrá 2 * num_workers lotes 
    # auto_find_batch_size=True,  # Encuentra automáticamente el tamaño de lote: si es True, busca automáticamente el tamaño de lote más 
                                # grande que se puede utilizar sin exceder la memoria GPU.
                                # Hay que tener accelerate instalado. Por defecto False.
    # torch_compile=True, # Optimización: Habilita la compilación de PyTorch para acelerar el entrenamiento.
    # torch_compile_backend="max-autotune", # Optimización: Utiliza la compilación de PyTorch para acelerar el entrenamiento.
    # torch_compile_mode="default",
    # neftune_noise_alpha=0.1, # Optimización: Ajusta el ruido de Neftune.
    # eval_on_start=True,  # Evalúa al inicio del entrenamiento: si es True, evalúa el modelo al inicio del entrenamiento (prueba que funcione correctamente).
    # use_liger_kernel=True # Optimización: Utiliza el kernel Liger para acelerar el entrenamiento.
    report_to=[]
)

In [21]:
from transformers import Trainer

trainer = Trainer(
    model=model,    # Modelo a entrenar. Puede ser uno preentrenado o uno personalizado (torch.nn.Module). 
                    # Si no se especifica, se debe usar `model_init` para inicializar el modelo.
    args=training_args, # Argumentos de entrenamiento: TrainingArguments que configuran el entrenamiento.
    data_collator=data_collator,    # Data collator: collator que se utiliza para agrupar los datos en lotes a partir de una
                                    # lista de elementos de train_dataset o eval_dataset. Por defecto es DefaultDataCollator,
                                    # si no se pasa processing_class.
    train_dataset=dataset['train'],  # Dataset de entrenamiento: dataset que se utiliza para el entrenamiento.
    eval_dataset=dataset['test'],    # Dataset de evaluación: dataset que se utiliza para la evaluación.
    processing_class=image_processor,  # Procesador de imágenes: procesador que se utiliza para pre-procesar las imágenes antes de pasarlas al modelo.
    # model_init=None,  # Inicializador de modelo: función que se utiliza para inicializar el modelo si no se pasa un modelo preentrenado. 
                        # Cada vez que se llama a `trainer.train()`, se inicializa un nuevo modelo (una nueva instancia). 
                        # Se suele utilizar para optimización. Puede tener un argumento que contiene un trial de Optuna o 
                        # Ray Tune, por ejemplo, para optimizar hiperparámetros. Esto permite modificar la arquitectura.
    # compute_loss_func=None, # Función de pérdida personalizada: función que se utiliza para calcular la pérdida durante el entrenamiento.
    compute_metrics=compute_metrics,    # Función de métricas personalizada: función que se utiliza para calcular las métricas durante la evaluación.
                                        # Debe recibir un objeto `EvalPrediction` y retornar un diccionario con las métricas calculadas.
    callbacks=callback_list,  # Lista de callbacks: lista de callbacks que se ejecutan durante el entrenamiento.
    # optimizers=(None, None),  # Optimizador y scheduler: tuplas que contienen el optimizador y el scheduler a utilizar. Por defecto AdamW.
    # optimizer_cls_and_kwargs=None,  # Clase y argumentos del optimizador: tupla que contiene la clase del optimizador y un 
                                    # diccionario con los argumentos a pasar al optimizador. Sobreescribe `optim` 
                                    # y `optim_kwargs` de `TrainingArguments`.
    # preprocess_logits_for_metrics=None,   # Preprocesador de logits para métricas: función que se utiliza para preprocesar los logits 
                                            # antes de calcular las métricas.
)

In [ ]:
# if __name__ == "__main__": # NOTA IMPORTANTE: Para cambiar el número de workers, se debe hacerlo en un .py y en el bloque `if __name__ == "__main__":`
# sino explota en Windows. Esto pasa en un notebook por ejemplo. En notebook dejar el número de workers en 0.
with mlflow.start_run():
    mlflow.log_param("dataset_name", DATASET_NAME)
    mlflow.log_param("dataset_version", DATASET_VERSION)

    logger.info("Iniciando entrenamiento del modelo...")
    trainer.train()
    logger.info("Entrenamiento finalizado. Guardando el modelo...")

history = pd.DataFrame(trainer.state.log_history)
history.to_csv(MODELS_DIR / MODEL_FOLDER / HISTORY_FILENAME, index=False)
history

🏃 View run adventurous-bear-539 at: http://localhost:8080/#/experiments/7/runs/838d9d99b3514894bf3f546555c6d924
🧪 View experiment at: http://localhost:8080/#/experiments/7


AttributeError: 'function' object has no attribute 'transforms_to_string'

In [ ]:
# Predicción con pipeline
from transformers import pipeline

classifier = pipeline('image-classification', model=model, image_processor=image_processor, device=DEVICE)
classifier(test_image)

Device set to use cuda


[{'label': 'Forest', 'score': 0.11400282382965088},
 {'label': 'SeaLake', 'score': 0.10881276428699493},
 {'label': 'AnnualCrop', 'score': 0.10688897967338562},
 {'label': 'PermanentCrop', 'score': 0.10486926883459091},
 {'label': 'River', 'score': 0.10069800168275833}]

In [ ]:
# Predicción con Pytorch
inputs = image_processor(test_image, return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs.to(DEVICE)).logits

predicted_label_index = logits.argmax(-1).item()
predicted_label = model.config.id2label[predicted_label_index]
print(f"Predicted label: {predicted_label} (index: {predicted_label_index})")

Predicted label: Forest (index: 1)
